<h1>Lab 1: Geoprocessing</h1>
<h3>Neighborhood Resurrection, Part 1: <em>Mapping a refugee route to Sudan in 1980s Ethiopia</em></h3>

<h5>Lab 1 Objective and Workflow:</h5>
<p>This script looks to clean, merge, layer, and append DEM</p>

<h5><strong>Data Sources:</strong></h5>
<ul>
    <li>Elevation -- Open Data Cube: Digtial Earth Africa API</li>
    <li>Conflict Events -- UCDP Ethiopia (1989)</li>
    <li>Subnational Administrative Boundaries</li>
</ul>

#### Imports

In [68]:
import geopandas as gpd
import pandas as pd
import csv
import json
import os
import pyproj
from dotenv import load_dotenv
import rasterio
import fiona
from osgeo import gdal
from shapely.geometry import Point

In [84]:
import xarray as xr
from odc.ui import DcViewer
from odc import algo

In [58]:
import requests
import time
import numpy as np
import matplotlib.pyplot as plt

In [67]:
import odc.geo.xr
from odc.geo.xr import xr_reproject
import psycopg2

In [3]:
from math import floor

### Importing Elevation Data from [OpenTopography](https://portal.opentopography.org/datasetMetadata?otCollectionID=OT.042013.4326.1)

In [81]:
# Connecting to Open Topography API
load_dotenv()
api_key = os.getenv('OpenTopographyKey')
dem_downloader = OpenTopographyDEM(api_key)

#### Manual Tiling

In [76]:
def tiled_elevation():
    api_key = api_key

    tiles = [
        # Row 1 - Southern tiles
        {'name': 'tile_sw', 'south': 7.5, 'north': 12.5, 'west': 31.0, 'east': 37.0},
        {'name': 'tile_se', 'south': 7.5, 'north': 12.5, 'west': 37.0, 'east': 41.0},
        # Row 2 - Northern tiles  
        {'name': 'tile_nw', 'south': 12.5, 'north': 17.5, 'west': 31.0, 'east': 37.0},
        {'name': 'tile_ne', 'south': 12.5, 'north': 17.5, 'west': 37.0, 'east': 41.0},
    ]

    downloaded_files = []

    for tile in tiles:
        params = {
            'demtype': 'SRTMGL1',
            'south': tile['south'],
            'north': tile['north'],
            'west': tile['west'],
            'east': tile['east'],
            'outputFormat': 'GTiff',
            'API_Key': api_key
        }

        response = requests.get(
            "https://portal.opentopography.org/API/globaldem",
            params=params
        )
        
        if response.status_code == 200:
            filename = f"{tile['name']}.tif"
            with open(filename, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded {filename}")
            downloaded_files.append(filename)
        else:
            print(f"Failed to download {tile['name']}: {response.status_code}")
        
        # Delay between requests
        import time
        time.sleep(1)
    
    return downloaded_files

In [64]:
tile_files = tiled_elevation()

Downloaded tile_sw.tif
Downloaded tile_se.tif
Downloaded tile_nw.tif
Downloaded tile_ne.tif


### Finding local UTM EPSG _based on midpoint between Khartoum and Addis Ababa_

In [4]:
## Finding local UTM EPSG based on midpoint between Khartoum and Addis Ababa
lon, lat = 35.688889, 12.313333

def utm_epsg_from_lonlat(lon, lat):
    zone = int(floor((lon + 180) / 6) + 1)
    if lat >= 0:
        epsg = 32600 + zone   # northern hemisphere
    else:
        epsg = 32700 + zone   # southern hemisphere
    return epsg

print("UTM EPSG:", utm_epsg_from_lonlat(lon, lat))

UTM EPSG: 32636


### Importing Conflict Data from [UCDP](https://ucdp.uu.se/encyclopedia), 1989-1991, _Sudan and Ethiopia_

In [14]:
sudan_conf = pd.read_csv('loaded_sudan_events.csv')
ethio_conf = pd.read_csv('loaded_ethiopia_events.csv')

In [ ]:
## filtering new df where 'year' is < 1995

In [15]:
ethio_conf['year'].unique()

array([2023, 2021, 2022, 2019, 2020, 2024, 2018, 2017, 2013, 2014, 2012,
       2001, 2002, 2003, 2004, 2010, 2011, 2006, 1989, 1990, 1991, 2007,
       2005, 2008, 2016, 1992, 1998, 1997, 1999, 1995, 2015, 2009, 2000,
       1996, 1994, 1993])

In [16]:
ethio = ethio_conf[ethio_conf['year'] <= 1995]

In [17]:
ethio['year'].unique()
ethio = ethio.sort_values(by='year')

In [18]:
sudan_conf['year'].unique()

array([2024, 2015, 2017, 2018, 2022, 2020, 2023, 2019, 2021, 2016, 2014,
       2013, 2011, 2006, 2007, 2005, 2008, 2002, 2004, 2003, 2001, 1993,
       1994, 1995, 1996, 2000, 1998, 1999, 1997, 2012, 2009, 2010, 1990,
       1989, 1991, 1992])

In [20]:
sudan = sudan_conf[sudan_conf['year'] <= 1995]

In [21]:
sudan['year'].unique()
sudan = sudan.sort_values(by='year')

In [22]:
sudan

,id,relid,year,active_year,code_status,type_of_violence,conflict_dset_id,conflict_new_id,conflict_name,dyad_dset_id,...,date_prec,date_start,date_end,deaths_a,deaths_b,deaths_civilians,deaths_unknown,best_est,high_est,low_est
4066,7999,SUD-1989-1-641-16,1989,True,Clear,1,309,309,Sudan: Government,663,...,1,10/17/1989 0:00:00,10/17/1989 0:00:00,0,0,0,12,12,12,12
4137,8189,SUD-1989-1-641-7,1989,True,Clear,1,309,309,Sudan: Government,663,...,1,03/28/1989 0:00:00,03/28/1989 0:00:00,6,0,0,76,82,82,82
4064,7920,SUD-1989-3-625-1,1989,True,Clear,3,112,480,Government of Sudan - Civilians,112,...,4,11/01/1989 0:00:00,11/30/1989 0:00:00,0,0,0,0,0,150,0
4236,8633,SUD-1989-1-641-12,1989,True,Clear,1,309,309,Sudan: Government,663,...,1,04/21/1989 0:00:00,04/21/1989 0:00:00,1,0,0,100,101,101,101
4136,8188,SUD-1989-1-641-14,1989,True,Clear,1,309,309,Sudan: Government,663,...,1,08/22/1989 0:00:00,08/22/1989 0:00:00,2,0,0,0,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2053,8109,SUD-1995-1-641-10,1995,True,Clear,1,309,309,Sudan: Government,663,...,4,09/16/1995 0:00:00,09/28/1995 0:00:00,0,0,0,5,5,5,5
1904,7658,SUD-1995-3-625-3,1995,True,Clear,3,112,480,Government of Sudan - Civilians,112,...,5,02/01/1995 0:00:00,04/30/1995 0:00:00,0,0,45,0,45,45,45
1844,7538,SUD-1995-1-641-1,1995,True,Clear,1,309,309,Sudan: Government,663,...,4,01/01/1995 0:00:00,01/09/1995 0:00:00,0,0,0,0,0,1000,0
4111,37289,UGA-1995-2-107-1.2,1995,True,Clear,2,5272,4662,LRA - SPLM/A,5272,...,4,10/01/1995 0:00:00,10/30/1995 0:00:00,18,0,0,0,18,18,18


In [23]:
sudan.to_csv('sudan.csv', index=False)

In [25]:
ethio.to_csv('ethio.csv', index=False)